# ArcticDEM Data Access

This notebook contains functions for querying available ArcticDEM strips and a demo of their use with an AOI near the Petermann Glacier in northern Greenland.

Learn more about the project at the [ArcticDEM overview page](https://www.pgc.umn.edu/data/arcticdem/).

Interactively explore and download PGC data with [FRIDGE](https://fridge.pgc.umn.edu/).

In [2]:
! pip install -q pdemtools

In [13]:
import zipfile
import os
import requests
from urllib.parse import urlparse
from pathlib import Path
import pprint
import geopandas as gpdworking_dir = Path.home() / "groundingline-data"


In [4]:
def create_data_directories(working_dir: Path | str) -> dict[[str], Path]:
    # Make sure the provided parent_dir exists
    if isinstance(working_dir, str):
        working_dir = Path(working_dir)
    working_dir.mkdir(parents=True, exist_ok=True)

    # Define the subdirectories to be created
    subdirs = {
        "index": working_dir / "arcticdem" / "index",
        "strip_dems": working_dir / "arcticdem" / "strip_dems",
        "mosaic_dems": working_dir / "arcticdem" / "mosaic_dems",
    }

    # Create the subdirectories
    for dir in subdirs.values():
        dir.mkdir(parents=True, exist_ok=True)

    return subdirs

In [5]:
def download_file(url: str, output_dir: Path | str) -> Path:
    """
    Downloads a file from the given URL to the specified directory.
    
    Args:
        url (str): The URL of the file to download.
        output_dir (str): The directory to save the file in.
    """

    
    # Convert the output_dir to a path object if it is not already and very that the directory exists
    if isinstance(output_dir, str):
        output_dir = Path(output_dir)
    if not output_dir.exists():
        raise ValueError(f"The provided output_dir does not exist: {output_dir}")
    
    # Get the filename from the URL
    parsed_url = urlparse(url)
    filename = os.path.basename(parsed_url.path)
    
    # Construct the full path to the output file
    output_file = output_dir / filename

    # If the output file already exists, skip redownloading
    if output_file.exists():
        return output_file
    
    # Download the file
    response = requests.get(url, stream=True)
    
    # Save the file to disk
    with open(output_file, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    
    return output_file

In [6]:
def extract_index_parquet(zip_path: str | Path) -> Path:
    """
    Unzips a zip archive containing one file to the same directory as the zip archive.

    Args:
        zip_file_path (str): The path to the zip archive file.
    """
    # Get the directory where the zip file is located
    zip_path = zip_path if isinstance(zip_path, Path) else Path(zip_path)
    directory = zip_path.parent

    # Open the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        # Get the name of the file inside the zip archive
        zip_members = zip_file.namelist()
        for member in zip_members:
            if ".parquet" in member:
                # Extract the file to the same directory as the zip archive
                unzipped = zip_file.extract(member, directory)
                
    return Path(unzipped)

In [7]:
def download_arcticdem_strip_index(url: str, index_dir: Path) -> Path:
    # Download the zip archive to the index subdirectory and extract it
    zip_archive = download_file(url=url, output_dir=index_dir)
    extracted = extract_index_parquet(zip_path=zip_archive)

    # Clean up after ourselves by: 
    #     - Moving the index file into the index directory
    #     - Delete the empty subdirectory created during zip extraction
    #     - Remove the zip archive
    strip_index = extracted.rename(index_dir / extracted.name)
    if extracted.parent.exists():
        extracted.parent.rmdir()
    if zip_archive.exists():
        zip_archive.unlink()

    return strip_index

In [8]:
ARCTICDEM_STRIP_INDEX_URL = "https://data.pgc.umn.edu/elev/dem/setsm/ArcticDEM/indexes/ArcticDEM_Strip_Index_latest_gpqt.zip"

working_dir = Path.home() / "groundingline-data"

In [9]:
dirs = create_data_directories(working_dir)

In [10]:
pprint.pprint(dirs)

{'index': PosixPath('/home/jovyan/groundingline-data/arcticdem/index'),
 'mosaic_dems': PosixPath('/home/jovyan/groundingline-data/arcticdem/mosaic_dems'),
 'strip_dems': PosixPath('/home/jovyan/groundingline-data/arcticdem/strip_dems')}


In [11]:
%%time
strip_index_path = download_arcticdem_strip_index(url=ARCTICDEM_STRIP_INDEX_URL, index_dir=dirs["index"])

CPU times: user 6.41 s, sys: 1.75 s, total: 8.16 s
Wall time: 1min 59s


In [12]:
strip_index_path

PosixPath('/home/jovyan/groundingline-data/arcticdem/index/ArcticDEM_Strip_Index_s2s041.parquet')

In [14]:
AOI_PATH = working_dir / "petermann_geom.json"

In [15]:
aoi = gpd.read_file(AOI_PATH)

In [17]:
aoi.explore()

In [21]:
type(aoi.geometry[0])

shapely.geometry.polygon.Polygon

In [18]:
import pdemtools as pdt

In [ ]:
strips = pdt.search(
    index_fpath=strip_index_path,
    bounds=aoi.geometry[0]
                   )